## Install packages

In [ ]:
!poetry install

## OPENAI settings

In [4]:
import os
os.environ['OPENAI_API_KEY'] = <openai-api-key>

## LLM

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

## Prompt template

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)